In [1]:
import pandas as pd

In [2]:
DATA_DIR = '/Users/asvs/kartheek_hobby_projects/loan_approval_prediction/data'
df_train = pd.read_csv(f'{DATA_DIR}/train.csv')
df_test  = pd.read_csv(f'{DATA_DIR}/test.csv')
df_sub = pd.read_csv(f'{DATA_DIR}/sample_submission.csv')

display(df_train.head(1))
print(df_train.shape)
display(df_test.head(1))
print(df_test.shape)


,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0


(58645, 13)


,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,58645,23,69000,RENT,3.0,HOMEIMPROVEMENT,F,25000,15.76,0.36,N,2


(39098, 12)


In [3]:
df_train.columns

Index(['id', 'person_age', 'person_income', 'person_home_ownership',
       'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt',
       'loan_int_rate', 'loan_percent_income', 'cb_person_default_on_file',
       'cb_person_cred_hist_length', 'loan_status'],
      dtype='object')

In [4]:
CATEGORICAL_COLS = ['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file']
NUMERICAL_COLS = ['person_age', 'person_income', 'person_emp_length', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length']
TARGET_COL = 'loan_status'